**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 

\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function **act** returns the next action to do. It firstly generates a random number between 0 and 1. If this number is lower than **epsilon**, the next action is chosen randomly (Exploration), and if this number is higher than **epsilon**, the action is chosen thanks to the policy learned by reinforcement of the agent (Exploitation).

**Epsilon** determines the rate of randomly selected actions (random exploration). 

This random exploration is essential. In fact, the agent needs to explore its environment in order to discover the best actions to do. If not, it will always do the best action learned until now and will never learn new behaviors that could be better.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # THERE WAS A MISTAKE HERE

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # THERE WAS A MISTAKE HERE
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=5 # set small when debugging
epochs_test=5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The arrays **position** and **board** represent the island on which the rat is running. They are initialized by a square of side equal to **grid_size** filled with zeros. On each side, two lines (or columns) of -1 are added. 

The array **position** is used to represent the path of the rat. The element whose coordinates are equal to those of the rat's position is set to 1 over iterations.

The array **board** is used to represent the rewards. The elements whose coordinates are equal to those of the cheese are set to 0.5 and the elements whose coordinates are equal to those of the poison are set to -1.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(4)
        return a


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state) # Not act here because no function act in the RandomAgent class 
                                              # No additional random exploration of epsilon

            # Apply an action to the environment, get the next state, the reward
            # and if the games end    
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/8.0. Average score (1.0)
Win/lose count 10.0/15.0. Average score (-2.0)
Win/lose count 10.0/11.0. Average score (-1.6666666666666667)
Win/lose count 9.5/19.0. Average score (-3.625)
Win/lose count 11.0/12.0. Average score (-3.1)
Final score: -3.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




(a) We have : 

$$\begin{align*}
  Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
             &= E_{p^{\pi}}[r(s_{0},a_{0}) + \sum_{t=1}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
             &= E_{p^{\pi}}[r(s,a) + \gamma \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \\
             &= E_{p^{\pi}}[E_{p^{\pi}}[r(s,a) + \gamma \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a] \\
             &= E_{p^{\pi}}[r(s,a) + \gamma E_{p^{\pi}}[ \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a] \\
             &= E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma E_{p^{\pi}}[ \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']] \\
   Q^\pi(s,a) &= E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^\pi(s',a')] \\
\end{align*}$$

This equation is named the Bellman's equation.


(b) We have : $Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$. 

We maximize the previous result with respect to $\pi$. However, as the only variable controled by $\pi$ is $a'$, this is equivalent to maximize the second term with respect to $a'$.

We have : 
$$\begin{align*}
    Q^*(s,a) &= r(s,a) + \gamma E_{(s^\prime, a^\prime)\sim p^{\pi^*}(.|s,a)} [\max_{a^\prime}Q^*(s^\prime,a^\prime)]\\
    Q^*(s,a) &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align*}$$


(c) Minimizing the loss $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ is equivalent to find the optimal $Q$-function ($Q^*(s,a)$). In fact, we are looking for the optimal $Q$-function by looking for the fixed point of the previous equation.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) >= self.max_memory:
            del self.memory[0]
        self.memory.append(m)

    def random_access(self):
        l = len(self.memory)
        return self.memory[np.random.randint(l)]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning (gamma)
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model.predict(s.reshape(1,5,5,self.n_state)).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape(1, 5, 5, self.n_state))
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount*self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(16, input_dim=20, activation='relu'))
        #model.add(Dense(12, activation='relu'))
        model.add(Dense(4, activation='softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [30]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0012 | Win/lose count 2.0/6.0 (-4.0)
Epoch 001/020 | Loss 0.0026 | Win/lose count 4.5/11.0 (-6.5)
Epoch 002/020 | Loss 0.0004 | Win/lose count 1.5/7.0 (-5.5)
Epoch 003/020 | Loss 0.0025 | Win/lose count 0.5/3.0 (-2.5)
Epoch 004/020 | Loss 0.0187 | Win/lose count 6.0/3.0 (3.0)
Epoch 005/020 | Loss 0.0156 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/020 | Loss 0.0010 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/020 | Loss 0.0092 | Win/lose count 3.0/5.0 (-2.0)
Epoch 008/020 | Loss 0.0111 | Win/lose count 1.5/2.0 (-0.5)
Epoch 009/020 | Loss 0.0145 | Win/lose count 4.5/8.0 (-3.5)
Epoch 010/020 | Loss 0.0336 | Win/lose count 4.5/7.0 (-2.5)
Epoch 011/020 | Loss 0.0071 | Win/lose count 4.5/12.0 (-7.5)
Epoch 012/020 | Loss 0.0013 | Win/lose count 3.5/9.0 (-5.5)
Epoch 013/020 | Loss 0.0147 | Win/lose count 6.0/5.0 (1.0)
Epoch 014/020 | Loss 0.0021 | Win/lose count 2.0/2.0 (0.0)
Epoch 015/020 | Loss 0.0119 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/020 | Loss 0.0012 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, 2, activation='relu', input_shape=(5,5,self.n_state)))
        model.add(Conv2D(32, 2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [32]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0031 | Win/lose count 0.5/5.0 (-4.5)
Epoch 001/020 | Loss 0.0210 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/020 | Loss 0.0067 | Win/lose count 4.5/1.0 (3.5)
Epoch 003/020 | Loss 0.0094 | Win/lose count 5.0/0 (5.0)
Epoch 004/020 | Loss 0.0037 | Win/lose count 3.0/1.0 (2.0)
Epoch 005/020 | Loss 0.0019 | Win/lose count 6.0/4.0 (2.0)
Epoch 006/020 | Loss 0.0013 | Win/lose count 2.5/4.0 (-1.5)
Epoch 007/020 | Loss 0.0642 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/020 | Loss 0.0077 | Win/lose count 4.5/4.0 (0.5)
Epoch 009/020 | Loss 0.0023 | Win/lose count 2.5/0 (2.5)
Epoch 010/020 | Loss 0.0012 | Win/lose count 3.5/1.0 (2.5)
Epoch 011/020 | Loss 0.0517 | Win/lose count 3.0/1.0 (2.0)
Epoch 012/020 | Loss 0.0018 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/020 | Loss 0.0065 | Win/lose count 3.5/5.0 (-1.5)
Epoch 014/020 | Loss 0.0031 | Win/lose count 3.0/0 (3.0)
Epoch 015/020 | Loss 0.0041 | Win/lose count 5.0/1.0 (4.0)
Epoch 016/020 | Loss 0.0021 | Win/lose count 0.5/3.0 (-2.5)

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [47]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/1.0. Average score (0.5)
Win/lose count 1.0/0. Average score (0.75)
Win/lose count 0.5/1.0. Average score (0.3333333333333333)
Win/lose count 1.5/0. Average score (0.625)
Win/lose count 0.5/2.0. Average score (0.2)
Win/lose count 1.5/1.0. Average score (0.25)
Win/lose count 0.5/1.0. Average score (0.14285714285714285)
Win/lose count 0.5/0. Average score (0.1875)
Win/lose count 2.5/1.0. Average score (0.3333333333333333)
Win/lose count 3.0/4.0. Average score (0.2)
Win/lose count 0.5/1.0. Average score (0.13636363636363635)
Win/lose count 0/0. Average score (0.125)
Win/lose count 0.5/0. Average score (0.15384615384615385)
Win/lose count 0/0. Average score (0.14285714285714285)
Win/lose count 0.5/0. Average score (0.16666666666666666)
Win/lose count 1.5/4.0. Average score (0.0)
Win/lose count 2.0/3.0. Average score (-0.058823529411764705)
Win/lose count 0.5/0. Average score (-0.027777777777777776)
Win/lose count 2.0/1.0. Average score (0.0263157894736842

In [48]:
HTML(display_videos('cnn_test10.mp4'))

In [49]:
HTML(display_videos('fc_test10.mp4'))

(a) When studying the performance of the algorithms, we can see that both convolutional and fully connected models perform poorly. In fact, it appears that the agent tends to get stuck in a portion of the grid.
This can be due to a lack of exploration, because epsilon is fixed, but also because there is no penalty for staying at the same place. Sometimes, it is thus better to stay in the same zone of the map than risking negative rewards by moving.

Hewever, generally, the convolutional model is more efficient than the fully connected model.


(b) When studying the influence of the temperature, it appears that the scores increase with the temperature. In fact, the higher the temperature is, the most positive rewards there are on the grid. Thus, at the beginning, the agent can find a lot of positive rewards around him. However, the same problem remain for every temperature, and the agent is not exploring the map deeply. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [12]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Decreasing epsilon over iterations
        agent.set_epsilon(agent.epsilon*0.9)

        while not game_over:            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        #self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # THERE WAS A MISTAKE HERE

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        
        if train:
            reward = - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
    
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # THERE WAS A MISTAKE HERE
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
       
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    



In [16]:
# Parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0157 | Win/lose count 10.0/26.900000000000038 (-16.900000000000038)
Epoch 001/020 | Loss 0.0139 | Win/lose count 12.0/19.80000000000002 (-7.8000000000000185)
Epoch 002/020 | Loss 0.0060 | Win/lose count 10.0/20.80000000000004 (-10.80000000000004)
Epoch 003/020 | Loss 0.0143 | Win/lose count 6.5/23.800000000000075 (-17.300000000000075)
Epoch 004/020 | Loss 0.0065 | Win/lose count 10.5/17.299999999999976 (-6.799999999999976)
Epoch 005/020 | Loss 0.0077 | Win/lose count 19.5/16.29999999999997 (3.2000000000000313)
Epoch 006/020 | Loss 0.0222 | Win/lose count 6.5/20.300000000000022 (-13.800000000000022)
Epoch 007/020 | Loss 0.0082 | Win/lose count 25.0/16.299999999999976 (8.700000000000024)
Epoch 008/020 | Loss 0.0356 | Win/lose count 15.5/17.499999999999986 (-1.9999999999999858)
Epoch 009/020 | Loss 0.0071 | Win/lose count 6.0/19.20000000000001 (-13.20000000000001)
Epoch 010/020 | Loss 0.0078 | Win/lose count 30.0/10.399999999999979 (19.600000000000023)
Epoch 011/020

In [17]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 13.0/0. Average score (13.0)
Win/lose count 17.0/0. Average score (15.0)
Win/lose count 11.5/1.0. Average score (13.5)
Win/lose count 4.5/0. Average score (11.25)
Win/lose count 10.5/0. Average score (11.1)
Win/lose count 9.0/4.0. Average score (10.083333333333334)
Win/lose count 10.5/1.0. Average score (10.0)
Win/lose count 2.0/0. Average score (9.0)
Win/lose count 28.5/0. Average score (11.166666666666666)
Win/lose count 14.5/0. Average score (11.5)
Win/lose count 15.5/1.0. Average score (11.772727272727273)
Win/lose count 11.5/0. Average score (11.75)
Win/lose count 19.5/2.0. Average score (12.192307692307692)
Win/lose count 19.0/1.0. Average score (12.607142857142858)
Win/lose count 23.0/0. Average score (13.3)
Win/lose count 11.0/0. Average score (13.15625)
Win/lose count 4.0/0. Average score (12.617647058823529)
Win/lose count 19.5/0. Average score (13.0)
Win/lose count 18.0/2.0. Average score (13.157894736842104)
Win/lose count 11.5/0. Average score (13.075)
Final

Better results are obtained!

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***